In [1]:
import utils
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from scipy.stats import norm


In [2]:
ratings,port ,yc,trans = utils.upload_data()

In [3]:
trans.head()

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,0.89810,0.09346,0.00547,0.00135,0.00072,0.00050,0.00030,0.00010
AA,0.00499,0.90725,0.08081,0.00490,0.00092,0.00060,0.00031,0.00021
A,0.00031,0.01672,0.92573,0.05225,0.00301,0.00109,0.00053,0.00034
BBB,0.00019,0.00096,0.03442,0.91899,0.03783,0.00443,0.00186,0.00134
BB,0.00011,0.00033,0.00121,0.05004,0.85576,0.07476,0.01144,0.00635


In [4]:

def CDS_Price(N, coup, spread,t2m,ye,freq):
    D = (1/(1+ ye/freq)**(freq*t2m))
    result =  N *  (spread - coup )/ ye *  (1-D)
    return(result)
def beta_recovery(ratings,alpha=1.62, beta=1.86):
    num_of_companies = ratings.shape[0]
    recoveries=np.random.beta(alpha,beta, (num_of_companies, 1))
    ratings["Recovery"] = recoveries
    return(ratings)

ratings_w_recov = beta_recovery(ratings)

In [5]:

port["Unique_id"] = np.arange(len(port))
port.head()


,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id
Name,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,146.504,0.033350,146.533443,40,3
Riocan Real Estate Investment,Bond,766910AW3,7,2024-04-17,0.037250,2,104.860,0.015597,104.859058,40,4


In [6]:
def yc_spread(yc):
    # Find spread between GOVernment and rating
    Wo_Gov =  yc.loc[:, yc.columns != "Government"]
    Wo_Gov_1 = Wo_Gov.loc[:, Wo_Gov.columns != "Tenor"]
    output = Wo_Gov_1 - np.array(yc.loc[:, yc.columns == "Government"])
    return(output)

In [7]:
ratings.set_index("Name",inplace=True)

In [8]:
yc_spreads = yc_spread(yc)
yc_spreads.head()

,AAA,AA,A,BBB,BB,B,CCC
In years,,,,,,,
0.25,0.000750,0.000900,0.001250,0.003550,0.010360,0.011070,0.024635
0.50,0.000270,0.000400,0.000740,0.003020,0.010190,0.011050,0.024820
1.00,0.000620,0.000710,0.001020,0.003250,0.011100,0.012290,0.026525
2.00,0.000480,0.000590,0.000920,0.003300,0.012930,0.014760,0.030255
3.00,0.000515,0.000625,0.001015,0.003675,0.014735,0.016995,0.033625


In [9]:
def Correlation_Matrix(ratings,port):
    """
    Correlation_Matrix consume  a pd.DataFrame of portfolio positions
    and returns the associated correaltion matrix for each instrument

    Remark: 

    At the  moment correaltion matrix is filled on the diagnonal with 1s and
    0.36 The code will be changed at a later date
    """
    Comp_ = pd.unique(port.index)
    #print(Comp_)
    n_assets = len(Comp_) 
    
    Cor_Matrix = np.diag([1 for i in range(n_assets)])
    
    Cor_matrix_pd =  pd.DataFrame(Cor_Matrix, index = Comp_, columns=Comp_)
    # Slow Method 
    for comp1 in Cor_matrix_pd.index:
        correlation = 0
        for comp2 in Cor_matrix_pd.columns:
            if comp1 == comp2:
                correlation = 1 
            elif ratings.loc[comp1, "Industry"] == ratings.loc[comp2, "Industry"]:
                correlation = 0.65
            else: 
                correlation = 0.28
            Cor_matrix_pd.loc[comp1,comp2] = correlation
    
    
    return(Cor_matrix_pd)


In [10]:
covar= Correlation_Matrix(ratings,port)

In [11]:
port.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id
Name,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,146.504,0.033350,146.533443,40,3
Riocan Real Estate Investment,Bond,766910AW3,7,2024-04-17,0.037250,2,104.860,0.015597,104.859058,40,4


In [12]:
port_rating = pd.merge(port, ratings_w_recov, on="Name")

In [13]:
port_rating

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery
Name,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.592278
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.424542
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.424542
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.424542
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.424542
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,37,BB,Energy,0.424542
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,38,BB,Energy,0.424542
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,146.504,0.033350,146.533443,40,3,BBB,Telecom,0.325887
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,145.696,0.033804,145.725011,40,15,BBB,Telecom,0.325887


In [14]:
def time_maturity(portfolio,start_date = date(2021,1,1), time_lead=0):
    """
        time_maturity consumes a pd.DataFrame representing the portfolio and a datetime object called 'start_date'
        and returns a portfolio pd.DataFrame with 'T2M' column representing the "Time to Maturity"

        time_maturity: pd.DataFrame, datetime.date -> pd.DataFrame
    """

    portfolio["T2M"] = portfolio["Maturity Date"].apply(lambda x: (relativedelta(x,start_date).years +(relativedelta(x,start_date).months/12))) - time_lead
    portfolio["T2M"] = np.where(portfolio["T2M"] <= 0, portfolio["T2M"] + 1, portfolio["T2M"]) # checks for negative dates,  and adds a year.
    return(portfolio)

In [15]:
def adjust_yield_curve(port,yc, time_lead=0):
    col_names = yc.columns 
    col_names.drop(["Government"])
    ret = port.copy()
    time_now = date(2021,1,1)
    
    ret = time_maturity(ret, time_now, time_lead) 
    
    #result = pd.DataFrame(columns=col_names)
    
    ### Adjusts yield _cur
    df =  yc.copy()
    for i in ret["T2M"]:
        if i not in yc.index:
            df.loc[i] = np.nan
   
    df.sort_index(axis= 0,inplace = True)
    df = df.interpolate(method="linear",axis=0) #new yc with times in between time to maturities.
    df.fillna(axis= 0, method="bfill", inplace = True) 
    # back fills rates for short term rates
    df = df[df.index.notnull()] # removes na indexes
    return(df)

In [16]:
### Part 2 of PART 1 code here
def market_val_port(port):
    l = port.Price * port.Notional /100
    l.columns = ["Name","Portfolio Value"]
    return(l)

def routine_price(portfolio,yc):
    # Set index in years to help make it easier for interpolation
    yc = yc.set_index("In years")
    """
    Description:
    Routine_price simulates the values of each bond and CDS relative to change in credit rating returning
    the simulated prices based on a transition matrix, and yield associated with the rating.

    Purpose:
    Using this procedure will reflect the price of the financial instrument based on credit rather for each credit rating.
    after a 1 year time step
    Variates:
    Routine_price: 
    - objtype: pd.DataFrame 
    - dtype: pd.float64
    - Matrix containing prices for each instrument.
    - Columns: The credit rating
    - rows: The  instrument and its prices at every Rating after passing 1 year
    
    portfolio: 
    - objtype: pd.DataFrame
    - dtype: pd.float64
    - Matrix containing portfolio of financial instruments and their associated information.
    
    yc:
    - objtype: pd.DataFrame
    - dtype: pd.float64
    - Matrix containing yields of bonds with respect to Credit ratings.
    """
    ### Todays date
    now_w = datetime.datetime.now()
    ## TODO Create routine such that for defaults result in Recovery value for Bonds, and Notional * recovery %
    # def makehams_formula(coup, rate, time_remaining, freq, Notional):
    #     ye = math.floor(time_remaining)
    #     adj = time_remaining - ye
    #     ## TODO adjust adj  such that we only need to consider the period after coupon.
        
    #     if adj > 0.5:
    #         time_to_maturity = ye + 0.5 
    #         adj  = (adj- 0.5)/0.5
    #     else:
    #         time_to_maturity = ye 
    #         adj = adj / 0.5
    #     # D is the discounting factor
    #     D = (1/(1+ rate/freq)**(freq* time_to_maturity))
    #     # P is the Clean price of the bond at the specific 
    #     Dirty = coup/rate * Notional *(1 - D)  + Notional * D
    #     # Dirty_price represents the Dirty Price of the bond
    #     P = Dirty * (1 + rate/2)**adj  -   Notional*coup*adj
    #     return(P, Dirty) 




    def makehams_formula(coup, rate, time_remaining, freq, Notional):
        ye = np.floor(time_remaining) 
        adj = time_remaining - ye
        t2 = ye.copy()
        print(adj)
        ## TODO adjust adj  such that we only need to consider the period after coupon.
        def check_adj(adj):
            if adj > 0.5:
                #time_to_maturity = ye + 0.5 
                adj  = (adj- 0.5)/0.5
            else:
                
                adj = adj / 0.5
            return(adj)
        def check_adj_2(adj):
            if adj > 0.5:
                #time_to_maturity = ye + 0.5 
                k =  0.5
            else:
                k = 0
            return(k)
        t1 = adj.copy()
        adjust = adj.apply(lambda x: check_adj(x)) 
        time_to_maturity = ye + t1.apply(lambda x: check_adj_2(x))

        # D is the discounting factor
        D = (1/(1+ rate/freq)**(freq*time_to_maturity))
        # P is the Clean price of the bond at the specific 
        Dirty = (coup/rate * Notional *(1 - D)  + Notional * D)
        # Dirty_price represents the Dirty Price of the bond
        P = Dirty * (1 + rate/2)**adjust  -   Notional*coup*adjust
        return(pd.from_dict({"Clean Price":P,"Dirty: Price" : Dirty },dtype= np.float64)) 
    
    # time_1 = now_w + timedelta(365.25)
    def CDS_pricer(status,Instrument,recovery,Notional):
        if Instrument == "CDS":
            if  status == "Default":
                return(Notional * recovery)
            else:
                return(0)
    
    
    

#df.loc[df['set_of_numbers'] <= 4, 'equal_or_lower_than_4?'] = 'True' 
#df.loc[df['set_of_numbers'] > 4, 'equal_or_lower_than_4?'] = 'False'
            

    def run_pricer(port):
    # TODO create a condition for CDSs and one for Bonds
    # I've created a column denoting CDS's
        #port.loc[df["Instrument type (Bond or CDS)"] == "Bond", 'Instrument Price'] 
        #port = price_CDS(port)
        #port = price_bonds(port)
        #return(port)
        pass

    
    # def one_step(port, rating):
    col_names = yc.columns
    col_names.remove("government","In years")
    ret = portfolio.copy()

    ret["status"]= ret["Instrument type (Bond or CDS)"].apply(lambda x: 1 if x == "Bond" else 0 )
    ret["Years_Remaining"] =  ret["Maturity_Date"].apply(lambda x: (relativedelta(x,now_w).years +(relativedelta(x,now_w).months/12))) - 1
    ret["Years_remaining"] = np.wherse(ret["Years Remaining"] <= 0, ret["Years Remaining"] + 1, ret["Years Remaining"])
    result = pd.DataFrame(columns=col_names)
    
    df =  yc.copy()
    for i in ret["Years_Remaining"]:
        if i not in yc.index:
            df.loc[i] = np.nan
   
    df.sort_index(axis= 0,inplace = True)
    df = df.interpolate(method="linear",axis=0) #new yc with times in between time to maturities.
    df.fillna(axis= 0, method="bfill", inplace = True) 
    # back fills rates for short term rates
    df = df[df.index.notnull()] # removes na indexes
    
    
    # Interporlate each rate. linear
    
    #ret["Instrument"] = ret["Instrument type (Bond or CDS)"].apply(lambda x: 1 if x  == "Bond")
    #TODO check if "In years" contains else interperolate

    pass

In [51]:
yc_1 = adjust_yield_curve(port,yc)
temp_yc_1year_fws = adjust_yield_curve(port,yc,time_lead=1)

In [19]:
port2 = time_maturity(port_rating)
port2.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M
Name,,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.592278,11.250000
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.424542,16.416667
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.424542,6.000000
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.424542,16.416667
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.424542,6.000000


In [49]:
def find_comp_spread(port, yc):
    """
        find_comp_spread coonsumes a portfolio port, and yield curve 'yc' and
        and returns the company specific spread and the ratings spread assocaited with the
        asset.

        --------- Paramaters ---------
        port: Portfolio
        type: pd.DataFrame

        yc: Yield curve
        type: pd.DataFrame
        disclaimer: Ensure it has been interpolated.

        Overall assumptions:

        1. Coupons listed for CDS are the fixed swap rates.
        2. In doing so for (1) our Ispreads represent the spread between Grade Rate and the coupon
        3. THe results were most consistant and relatively properly evaluted.
    """
    temp = port.copy()
    temp["Grade rate"] = yc.lookup(temp["T2M"], temp["Issuer Rating"])
    temp["ISpread"] = np.where(temp["Instrument type (Bond or CDS)"]=="Bond", 
        temp["Yield"] - temp["Grade rate"], 
        temp["Coupon"] - temp["Grade rate"])
    return(temp)

In [53]:
temp2 = find_comp_spread(port2,yc_1)
temp2.head()

/Users/rickyyung/anaconda3/envs/generative/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.


,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate,ISpread
Name,,,,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.592278,11.250000,0.024755,0.002849
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.424542,16.416667,0.052206,0.016080
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.424542,6.000000,0.027982,0.016033
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.424542,16.416667,0.052206,0.017196
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.424542,6.000000,0.027982,0.017149


# Dealing with Recovery

In [26]:
def beta_recovery(ratings,alpha=1.62, beta=1.86):
    num_of_companies = ratings.shape[0]
    recoveries=np.random.beta(alpha,beta, (num_of_companies, 1))
    ratings["Recovery"] = recoveries
    return(ratings)
    
    

In [27]:
def calculate_recovery(recov, Notional):
    """
        calculate_recovery consumes a pd.series/ numpy.narray1d reprsenting the recoveries in 
        in percentages and returns a new column of the beta randomly generated recoveries.
    """
    return(recov* Notional)

In [28]:
def CDS_Price(N, coup, spread,t2m,ye,freq):
    D = (1/(1+ ye/freq)**(freq*t2m))
    result =  N *  (spread - coup )/ ye *  (1-D)
    return(result)


In [29]:
# I want to split the data to two different portfolios 
# Scenario BREAKDOWN
def one_year_forward(port,transition,yc):
    """
        Purpose: 
        one_year_forward takes in a portfolio 'port', transition matrix transition, and yield curve 'yc'
        and returns a pandas.DataFrame with all the instrumetns and possible ratings and prices associated.
        
        one_year_forward ( pd.DataFrame, pd.DataFrame, pd.DataFrame) -> pd.DataFrame
        PARAMETERS:
        
        port: pd.DataFrame 
            desc: Holds a portfolio of fixed income instrumemnts
        
        transition: pd.DataFrame
            desc: Transition matrix of ratings
        
        yc: pd.DataFrame
            desc: yield curve

    Disclaimer: 
    One must use the yield curve that matches your future scenarios.
    Please run the adjust_yield_curve function with time_lead set to number of years ahead.
    Please give a fresh raw data portfolio. If T2M is already adjsuted it will not work as 
    intended.

    In addition, when Initially wrote this program I considered each instrument to have their own implied company
    Related Spread. This is because we have overalpping instruments with similary spreads but over different days
    """
    grades = np.unique(transition.columns)
    # Function begins
    temp = port.copy() # Take a copy of our portfolio ( Pandas sucks when we dont do this)
 
    temp["T2M_1yr"] = temp["T2M"] - 1  # Reduce the number of Time to Maturity by 1 year 
    # Set a column where T2M is one year less and if year becomes negative adjust back to 1 year
    temp["T2M_1yr"] = np.where(temp["T2M_1yr"] < 0 , 1 , temp["T2M_1yr"])

    desired_col = ["Unique_id", "Instrument type (Bond or CDS)", "T2M_1yr","Coupon","ISpread", "Coupons per year",
     "Notional","Recovery", "Yield"] # Desired columns for Pricing purposes
    
    grades_list = grades.tolist()
  
    result = temp[desired_col]
    result.reset_index(inplace=True)
    # Cross Merge Trick (shouldve used .melt but I am Lazy.)
    temp_grades_matrix  = pd.DataFrame({"New Ratings": grades_list})
    result_f = pd.merge(result, temp_grades_matrix, how="cross")

    # Reset the index for name
    result_f.set_index("Name", inplace=True)
    # For adjustment Purposes, which we will later address,  I will set the default yield to a very large number
    yc["D"] = 1e7
    
    # We look up our yield curve with our new ratings and we return new yield with simulated ratings
    # Sim Yields are the potential yields for each simulated rating.
    result_f["Sim Yields"] = yc.lookup(result_f["T2M_1yr"], result_f["New Ratings"])

    # Adjust Yields with new with Compnay and instrument specific spread
    result_f["New Yields"]  = result_f["ISpread"] + result_f["Sim Yields"]
    
    
    result_f["Sim Prices"] = np.where(result_f["Instrument type (Bond or CDS)"] == "Bond", utils.makehams_formula(
        result_f.Coupon, result_f["New Yields"], result_f["T2M_1yr"],result_f["Coupons per year"],result_f["Notional"]
    ), CDS_Price(result_f["Notional"],result_f["Coupon"],result_f["New Yields"],result_f["T2M_1yr"],result_f["Sim Yields"],result_f["Coupons per year"]))
    Step3 = result_f.copy()

    # TO ADDRESS the ISSUE we ran into before with the .lookup operation, we now have to adjsut back the pricing for defaulted bonds
    # Hence we must return Recovery * Notional and for CDS we return (1-Recovery) * Notional
    Step3["Sim Prices"] = np.where((Step3["New Ratings"] == "D") & (Step3["Instrument type (Bond or CDS)"] == "Bond"),
         calculate_recovery(Step3["Recovery"], Step3["Notional"]), Step3["Sim Prices"])
    Step3["Sim Prices"] =  np.where((Step3["New Ratings"] == "D") & (Step3["Instrument type (Bond or CDS)"] == "CDS"),
         calculate_recovery(1-Step3["Recovery"], Step3["Notional"]),Step3["Sim Prices"])


    return(Step3)

In [30]:
yc_1yr = adjust_yield_curve(temp2, yc,1)

In [32]:
scenarios_2 = one_year_forward(temp2 ,trans,yc_1yr)
scenarios_2.head()

/Users/rickyyung/anaconda3/envs/generative/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.


,Unique_id,Instrument type (Bond or CDS),T2M_1yr,Coupon,ISpread,Coupons per year,Notional,Recovery,Yield,New Ratings,Sim Yields,New Yields,Sim Prices
Name,,,,,,,,,,,,,
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,A,0.019698,0.022547,10.489044
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,AA,0.018163,0.021012,10.612466
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,AAA,0.017548,0.020397,10.662417
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,B,0.045610,0.048460,8.651052
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,BB,0.041690,0.044540,8.901643


In [33]:
scenarios = one_year_forward(temp2 ,trans,yc_1yr)
scenarios.head()

/Users/rickyyung/anaconda3/envs/generative/lib/python3.7/site-packages/ipykernel_launcher.py:56: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.


,Unique_id,Instrument type (Bond or CDS),T2M_1yr,Coupon,ISpread,Coupons per year,Notional,Recovery,Yield,New Ratings,Sim Yields,New Yields,Sim Prices
Name,,,,,,,,,,,,,
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,A,0.019698,0.022547,10.489044
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,AA,0.018163,0.021012,10.612466
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,AAA,0.017548,0.020397,10.662417
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,B,0.045610,0.048460,8.651052
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.592278,0.027605,BB,0.041690,0.044540,8.901643


# Simulating results

In [34]:
def look_for_rating(x, cdftrans, initial_rating):
    d= cdftrans.loc[[initial_rating]]
    look = d.T
    e=look[look[initial_rating].ge(x)].index[0]
    return(e)   

In [35]:

def ratings_sim(port,trans,covar,ratings,  N=2000):
    
    #TODO take a normal random draw with Cholesky L  then multiply and get new numbers and convert to letters
    n_comp = len(port.index.unique()) # Find Unique Companies
    # Reason we do this is to avoid unique random draws for each instrument but rather simply the company itself
    ############ SIMULATION BEGINS HERE ################
    # Randomly Normal Draw specifically  for those comapaniess
    normal_draws= np.random.standard_normal((n_comp, N)) 
    # Lower triangular for the covariance matrix.
    L =  np.linalg.cholesky(covar) 
    # after adjusting for correlation amongst asset returns.
    correlated_draws = norm.cdf(L.T @  normal_draws) 
    
    # TODO Now that we have correlated draws we now have to turn the 
    
    t_bounds =  trans[trans.columns[::-1]].cumsum(axis=1 ) # axis =1  is row_wise cumulative sum

    Step4  = pd.DataFrame(correlated_draws,index=port.index.unique() )    
    
    Step4 = Step4.join(ratings)

    # building table of ratings only for companies
    for Scenario in range(N):
        for name in Step4.index:
                try:
                    Step4.loc[name,Scenario] = look_for_rating(Step4.loc[name,Scenario], 
                    t_bounds, ratings.loc[name,"Issuer Rating"])
                except:
                    Step4.loc[name,Scenario] = "D"

    Step4 = Step4[[i for i in range(N)]]
    
    return(Step4)

In [36]:
s= ratings_sim(temp2,trans,covar,ratings,  N=2000)
s.head()

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
Name,,,,,,,,,,,,,,,,,,,,,
BELL CANADA,AA,BBB,BBB,A,BBB,AAA,A,BBB,BBB,BB,...,BBB,BBB,A,BBB,D,AA,BBB,A,D,BBB
Atlantic Power LP,BBB,BB,BB,BBB,BBB,AAA,AA,BB,BB,BB,...,B,BB,BBB,B,CCC,BBB,BB,BBB,BB,A
Rogers Communications Inc,BBB,BBB,BBB,BBB,BBB,A,BBB,BBB,BBB,BBB,...,BBB,BBB,BBB,BBB,BBB,A,BBB,BBB,BBB,BBB
Riocan Real Estate Investment,BBB,BBB,BBB,BBB,A,A,BBB,BBB,BBB,BBB,...,BBB,BBB,BBB,BBB,BBB,BBB,BBB,BBB,BBB,BBB
CANADIAN IMPERIAL BANK,BBB,BBB,BBB,AA,BBB,AAA,BBB,BBB,BBB,BBB,...,BB,BBB,AA,BBB,BBB,AA,BBB,BBB,BBB,A


# Blending pricing with scenario's generated

In [37]:
temp3 = temp2.copy()
temp3.reset_index(inplace=True)

temp3.head()


,Name,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate,ISpread
0,BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.592278,11.250000,0.024755,0.002849
1,Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.424542,16.416667,0.052206,0.016080
2,Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.424542,6.000000,0.027982,0.016033
3,Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.424542,16.416667,0.052206,0.017196
4,Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.424542,6.000000,0.027982,0.017149


In [38]:
def run_pricing(port,sim, lookup_table):
    """
        run_pricing pd.DataFrame,pd.DatFrame, pd.DataFrame -> pd.DataFrame
        run_pricing consumes a portfolio port, simulated results sim, and a scenario 
        lookup table lookup_table and returns the 
        prices of the instrument
    """
    # TODO loop through all scenarios, and check for where companies are same in name, then we can read the rating
    # and price using the rating and instrument read.
    output = pd.DataFrame(columns=[i for i in range(sim.shape[1])])

    for i in range(sim.shape[1]):
        temp_1 = pd.merge(port,sim[i],left_on="Name", right_on="Name")
        handle = temp_1.merge(lookup_table, left_on=["Name", "Instrument type (Bond or CDS)","Unique_id",i], right_on=["Name", "Instrument type (Bond or CDS)","Unique_id","New Ratings"])
        output[i] = handle["Sim Prices"]
    return(output)

        

In [39]:

def mkt_value_v2(port):
    """
    market_value is a function that takes a pd.DataFrame of portfolio and returns the market value
    market_value: pd.DataFrame ->  pd.DataFrame
    Remark: At the moment the theortical clean price is the price, we will make the assumption of the 
    the proper price later on
    """
    port["Market Value"] =  port["Notional"] * port["Price"] / 100
    Total = port["Market Value"].sum()
    print("Total portfolio Value is: {}".format(Total))

    return(port)

In [40]:
port_mkt = mkt_value_v2(temp3)


Total portfolio Value is: 217.18830000000003


In [41]:
MC_result = run_pricing(temp3, s, scenarios_2)
MC_result.head()


,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,10.612466,10.142645,10.142645,10.489044,10.142645,10.662417,10.489044,10.142645,10.142645,8.901643,...,10.142645,10.142645,10.489044,10.142645,4.145945,10.612466,10.142645,10.489044,4.145945,10.142645
1,7.869873,6.509541,6.509541,7.869873,7.869873,8.577709,8.478450,6.509541,6.509541,6.509541,...,6.269423,6.509541,7.869873,6.269423,4.850243,7.869873,6.509541,7.869873,6.509541,8.320657
2,-0.428267,-0.086907,-0.086907,-0.428267,-0.428267,-0.528106,-0.524467,-0.086907,-0.086907,-0.086907,...,-0.017943,-0.086907,-0.428267,-0.017943,0.412426,-0.428267,-0.086907,-0.428267,-0.086907,-0.509422
3,7.781057,6.440288,6.440288,7.781057,7.781057,8.478523,8.380725,6.440288,6.440288,6.440288,...,6.203565,6.440288,7.781057,6.203565,4.803982,7.781057,6.440288,7.781057,6.440288,8.225250
4,-0.428267,-0.086907,-0.086907,-0.428267,-0.428267,-0.528106,-0.524467,-0.086907,-0.086907,-0.086907,...,-0.017943,-0.086907,-0.428267,-0.017943,0.412426,-0.428267,-0.086907,-0.428267,-0.086907,-0.509422


In [42]:
MC_result.sum()

0       200.938204
1       197.266287
2       197.177032
3       200.914242
4       200.425092
           ...    
1995    203.363482
1996    197.450592
1997    200.814781
1998    191.453892
1999    201.620307
Length: 2000, dtype: float64

# FInding Statistics of Losses

In [43]:
VAR_95,VAR99, ES_98, _ =  utils.Calculate_Statistics(MC_result,port_mkt["Market Value"].sum())


In [44]:
VAR_95


-27.34377180411883

In [45]:
VAR99

-32.88298987710986

In [46]:
ES_98

-34.31676371903715